In [131]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
from pathlib import Path
from typing import List, Dict
import logging
import json

from pydantic import BaseModel, Field
from typing import List, Optional

from data_processing.xml_processing import wrap_lines, unwrap_lines, lines_from_wrapped_text
from data_processing.text_processing import get_text_from_file, write_text_to_file

In [133]:
from dp_video_processing import get_youtube_urls_from_file, download_audio_yt, detect_boundaries, split_audio_at_boundaries, process_audio_chunks, split_on_silence, postprocess_text

In [134]:
from data_processing.gpt_processing import token_count

In [135]:
# Configure main logger using setup_logger
import logging_config
from logging_config import setup_logging
from logging_config import get_child_logger

In [136]:
setup_logging(log_filename="audio_extract_testing.log", log_level=logging.DEBUG)
logger = get_child_logger("audio_extract_testing")

2024-12-10 17:47:53,175 - tnh.audio_extract_testing - DEBUG - Created logger with name: tnh.audio_extract_testing


In [137]:
audio_storage_dir = Path("processed_videos/video_transcriptions")

In [138]:
talk_name = "PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)"

In [139]:
# talk_name = "'Awakening Together' Family Retreat Dharma Talk ｜ Sister Kính Nghiêm ｜ 2024-06-27"

In [140]:
audio_working_dir = audio_storage_dir / talk_name
print(audio_working_dir)

processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)


In [141]:
audio_file_path = audio_working_dir / f"{talk_name}.mp3"
print(audio_file_path)

processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).mp3


In [142]:
transcript_path = audio_working_dir / f"{talk_name}.txt"
print(transcript_path)

processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).txt


In [143]:
audio_file_path.exists(), transcript_path.exists()

(True, True)

In [144]:
transcript_path = audio_working_dir / f"{talk_name}.txt"
print(transcript_path.name)

PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).txt


In [145]:
wrap_transcript_path = audio_working_dir / f"wrap_{talk_name}.txt"

In [171]:
section_output_path = audio_working_dir / f"section_{talk_name}.json"
print(section_output_path)

processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)/section_PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).json


In [147]:
post_output_path = audio_working_dir / f"format_{talk_name}.txt"

In [148]:
postprocess_section_vi = """You are a highly skilled and meticulous assistant processing an audio transcript of a Dharma Talk given in Vietnamese.
Each line of the transcript is numbered in the format: <NUM:LINE> 
You goal is to divide the entire transcript into logical sections based on content. 
For each section, give the title, both in Vietnamese and English, and starting and ending line numbers.
Also provide a brief summary of the text in English.
IMPORTANT: Every line in the transcript must belong to a section. Don't leave out any lines. Don't include lines in more than one section."""


### sectioning test messages:
"""You are a highly skilled and meticulous assistant processing an audio transcript of a Dharma Talk given in Vietnamese.
Each line of the transcript is numbered in the format: <NUM:LINE> 
You goal is to divide the entire transcript into logical sections based on content. 
For each section, give the title, both in Vietnamese and English, and starting and ending line numbers.
Also provide a brief summary of the text in English.
IMPORTANT: Every line in the transcript must belong to a section. Don't leave out any lines. Don't include lines in more than one section."""

You are a highly skilled and meticulous assistant processing an audio transcript of a Dharma Talk given in Vietnamese.
You goal is to divide the entire transcript into logical sections based on content. 
For each section insert a <section> tag, and close the section with </section> 
IMPORTANT: Every line in the transcript must belong to a section. Don't leave out any lines. Don't include lines in more than one section.

In [ ]:
instructions_translate_vi_1 = """You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

You are translating a section titled '{section_title}' from a Dharma talk offered by a Venerable Vietnamese Monastic at Deer Park Monastery in California.

Some transcriptions may be from sounds such as a bell. These can be marked as [Bell].

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a text that most closely matches the speaker's meaning,
while still giving clear and eloquent English.

Your goal is to translate the text and to correct errors (logical, transcription, or grammatical). 
You must faithfully capture the speaker's style and presentation while creating a meaningful flow using common, clear, and typical English. 
Translate faithfully and as carefully as possible. 

Do not leave out any content or summarize. 

The final output should match approximately the length and detail of the original.

Your output should be a polished section.

Make no other changes; add no content.

Output the final text only."""

In [ ]:
instructions_translate_vi_2 = """You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

You are translating a section titled '{section_title}' from a Dharma talk offered by a Venerable Vietnamese Monastic at Deer Park Monastery in California.

Lines of the transcript are numbered and are given in the format <NUM:LINE>.

Your task is to translate each line into correct, clear and typical English. Add correct punctuation to create meaning that matches the speakers style and intent.

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a text that most closely matches the speaker's meaning,
while still giving clear and eloquent English. Give the best approximation or contextual guess if the transcript is difficult or unclear. Make no comments.

You may consider adjacent lines for corrections and context when generating a line, however each line of translation should be as close as possible a translation of the original line.

Some transcriptions may be from sounds such as a bell. These can be marked as [Bell].

You must faithfully capture the speaker's style and presentation while creating a meaningful flow.

Do not leave out any content or summarize. 

The final output should match the same line structure as the original.

Your output should be a polished section.

Make no other changes; add no content.

Output the final text only."""

In [ ]:
instructions_translate_vi_3 = """You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

You are translating a section titled '{section_title}' from a Dharma talk offered by a Venerable Vietnamese Monastic at Deer Park Monastery in California.

Lines of the transcript are numbered and are given in the format <NUM:LINE>.

Your task is to translate each line into correct, clear and typical English. 

Add correct punctuation to create meaning that matches the speakers style and intent and creates flow between lines. You may adjust capitalization as needed for correctness.

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a text that most closely matches the speaker's meaning,
while still giving clear and eloquent English. Give the best approximation or contextual guess if the transcript is difficult or unclear. Make no comments.

You may consider adjacent lines for corrections and context when generating a line, however each line of translation should be based on the original line.

Some transcriptions may be from sounds such as a bell. These can be marked as [Bell].

You must faithfully capture the speaker's style and presentation while creating a meaningful flow.

Do not leave out any content or summarize. 

The final output should match the same line structure as the original.

Your output should be a polished section.

Make no other changes; add no content.

Output the final text only."""

You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

You are translating a section titled '{section_title}' from a Dharma talk offered by a Venerable Vietnamese Monastic at Deer Park Monastery in California.

Lines of the transcript are numbered and are given in the format <NUM:LINE>.

Your task is to translate each line into correct, clear and typical English.

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a text that most closely matches the speaker's meaning,
while still giving clear and eloquent English.

You may consider adjacent lines for corrections and context when creating a line, however each line of translation should be as close as possible a translation of the original line.

Some transcriptions may be from sounds such as a bell. These can be marked as [Bell].

You must faithfully capture the speaker's style and presentation while creating a meaningful flow.

Translate faithfully and as carefully as possible. 

Do not leave out any content or summarize. 

The final output should match the same line structure as the original.

Your output should be a polished section.

Make no other changes; add no content.

Output the final text only.

You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

You are translating a section titled '{section_title}' from a Dharma talk offered by a Venerable Vietnamese Monastic at Deer Park Monastery in California.

Lines of the transcript are numbered and are given in the format <NUM:LINE>.

Your task is to translate each line into correct, clear and typical English.

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a translation
that matches the speaker's meaning and intent within the context of the section, and is in creates a meaningful, logical and correct flow in English.

Consider adjacent lines and the context of the section for corrections when translating a line, however, each line of translation should be based on the original line.

Some transcription lines may be from sounds such as a bell. These can be marked as [Bell].
 
You must faithfully capture the speaker's style, presentation and intent while creating coherent, eloquent content across all lines.

Do not leave out any content or summarize. 

The final output should match exactly the same line structure as the original.

Make no other changes; add no content.

Output the final text only.

In [150]:
postprocess_format_en = """You are the world's leading expert at formatting Dharma talk audio transcriptions into written text for native, and partly fluent English speakers. 

The current text is from a Dharma Talk offered by a Southern Californian English-speaking monastic.

Make necessary corrections to grammar to create correct English sentence structure and logical flow. 

Insert <section> and <title> tags where appropriate in the text to mark natural sections in the talk; give these sections appropriate titles.

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a text that most closely matches the speaker's meaning in clear and eloquent English.

Your goal is to format the text into meaningful paragraphs and sections while correcting errors (logical, transcription, or grammatical). 

Faithfully to convey the speaker’s intended meaning as accurately as possible while maintaining the original tone and style. Use the speaker's original phrasing as much as possible.

Do not leave out any content. Do not summarize. 

Output the final text only."""

In [151]:
text = get_text_from_file(transcript_path)

In [152]:
wtext = wrap_lines(text, number=True)

In [153]:
print(wtext)

<1:Chuông>
<2:Chuông>
<3:Chuông>
<4:Nam Mô Đức Bộ Sưu Bụt Thích Ca Mô Đi>
<5:Kính Bạch Hiện Tiền Chư Tông Đức Tăng Ni>
<6:Kính Thưa Đại Chúng>
<7:Hôm nay là ngày 4 tháng 8, 2024>
<8:Có những câu chuyện người ta nói về thiền rất là thú vị>
<9:Đó là có một người thân chủ của một nhà hàng kia>
<10:Ông vô tiệm ăn đó>
<11:Người ta bưng lên những món ăn rất là ngon lành>
<12:Một bàn đầy>
<13:Thế rồi ông gọi cái người chủ tiệm lên>
<14:Ông nói là ông hãy nếm thử thức ăn của ông coi ra làm sao>
<15:Và cái vị chủ tiệm đó rất là kinh ngại>
<16:Thứ nhất là đây là một người thân chủ>
<17:Đến ăn ở đây rất là thường xuyên>
<18:Thứ hai đó là những đầu bếp của nhà hàng này>
<19:Là những đầu bếp giỏi nhất>
<20:Và chế tác những thức ăn rất là ngon>
<21:Thế nhưng mà hôm nay cái ông thân chủ này>
<22:Rất là người bạn quen thân này>
<23:Cũng gọi những món ăn thường ngày ông rất thích thú>
<24:Mà tại sao ông lại bắn dối như thế này>
<25:Thế rồi ông chủ nhà hỏi chứ anh không hài lòng về món chi>
<26:Có phải 

In [154]:
wtext_test = lines_from_wrapped_text(wtext, 1, 100)

In [161]:
wrap_transcript_path

PosixPath('processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)/wrap_PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).txt')

In [156]:
write_text_to_file(wrap_transcript_path, wtext)

/Users/phapman/Desktop/tnh-scholar/data_processing/text_processing/clean_parse_tag.py:150: FileExistsWarning: The file 'processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)/wrap_PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).txt' already exists. Use overwrite=True to overwrite.
  warnings.warn(f"The file '{full_path}' already exists. Use force=True to overwrite.", FileExistsWarning)


In [157]:
class Section(BaseModel):
    title_vi: str = Field(
        ..., 
        description="The title of the section in Vietnamese."
    )
    title_en: str = Field(
        ..., 
        description="The translation of the title of the section in English."
    )
    start_line: int = Field(
        ..., 
        description="The starting line number of this section."
    )
    end_line: int = Field(
        ...,
        description="The ending line number of this section."
    )

class DharmaTalkSections(BaseModel):
    talk_summary: str = Field(
        ..., 
        description="A brief summary of the Dharma talk content in English."
    )
    sections: List[Section] = Field(
        ..., 
        description="An ordered list of sections with their titles and included start and end line numbers."
    )

In [158]:
wrap_transcript_path

PosixPath('processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)/wrap_PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).txt')

In [160]:
print(postprocess_section_vi)

You are a highly skilled and meticulous assistant processing an audio transcript of a Dharma Talk given in Vietnamese.
Each line of the transcript is numbered in the format: <NUM:LINE> 
You goal is to divide the entire transcript into logical sections based on content. 
For each section, give the title, both in Vietnamese and English, and starting and ending line numbers.
Also provide a brief summary of the text in English.
IMPORTANT: Every line in the transcript must belong to a section. Don't leave out any lines. Don't include lines in more than one section.


In [162]:
section_object = postprocess_text(wrap_transcript_path, postprocess_section_vi, response_object=DharmaTalkSections, batch=False, max_tokens=5000)

2024-12-10 17:49:20,104 - tnh.video_processing - INFO - Starting postprocessing of file:
	processed_videos/video_transcriptions/PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)/wrap_PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).txt
2024-12-10 17:49:20,105 - tnh.gpt_interface - DEBUG - Starting chat completion with response_object=<class '__main__.DharmaTalkSections'> and max_tokens=5000...
2024-12-10 17:49:34,791 - tnh.video_processing - DEBUG - Full completion:
ParsedChatCompletion[DharmaTalkSections](id='chatcmpl-Ad68mKNXJ5L6Hv8xObM3EBVntUv0L', choices=[ParsedChoice[DharmaTalkSections](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[DharmaTalkSections](content='{"talk_summary":"In this Dharma Talk, the speaker explores the concepts of internal happiness, spiritual growth, and the cultivation of wisdom and compassion. The talk emphasizes that true happiness arises from 

In [238]:
section_object.sections

[Section(title_vi='Lời mở đầu', title_en='Introduction', start_line=1, end_line=7),
 Section(title_vi='Câu chuyện về khách hàng và chủ tiệm', title_en='The Story of the Customer and the Restaurant Owner', start_line=8, end_line=37),
 Section(title_vi='Bài học từ sự thiếu phương tiện', title_en='Lesson from Lack of Utensils', start_line=38, end_line=61),
 Section(title_vi='Hạnh phúc nội tại so với điều kiện bên ngoài', title_en='Inner Happiness versus External Conditions', start_line=62, end_line=91),
 Section(title_vi='Tầm quan trọng của sự phát triển tâm linh', title_en='Importance of Spiritual Development', start_line=92, end_line=155),
 Section(title_vi='Quá trình trưởng thành tâm thức', title_en='Process of Mind Maturity', start_line=156, end_line=184),
 Section(title_vi='Sự trưởng thành tình cảm', title_en='Emotional Maturity', start_line=185, end_line=202),
 Section(title_vi='Mở rộng lòng vị tha và trưởng thành trí thức', title_en='Expanding Compassion and Intellectual Maturity',

In [233]:
len(section_object.sections)

16

In [236]:
sections_translated = []
sections_original = []
sections = section_object.sections
section_range = range(0, 16)
backup_file = Path("backup_section_data.txt")
for i in section_range:
    section = sections[i]
    original_lines = lines_from_wrapped_text(wtext, section.start_line, section.end_line, keep_brackets=True)
    translate_instructions = instructions_translate_vi_2.format(section_title=section.title_en)
    logger.info(f"Translating section '{section.title_en}'...")

    if i == 0:
        logger.info(f"Translation instructions:\n{translate_instructions}")
    
    translated_lines = postprocess_text(original_lines, translate_instructions, batch=False)
    sections_translated.append(translated_lines)
    write_text_to_file(backup_file, translated_lines, append=True)
    sections_original.append(original_lines)



2024-12-10 21:18:53,904 - tnh.audio_extract_testing - INFO - Translating section 'Introduction'...
2024-12-10 21:18:53,905 - tnh.audio_extract_testing - INFO - Translation instructions:
You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

You are translating a section titled 'Introduction' from a Dharma talk offered by a Venerable Vietnamese Monastic at Deer Park Monastery in California.

Lines of the transcript are numbered and are given in the format <NUM:LINE>.

Your task is to translate each line into correct, clear and typical English. Add correct punctuation to create meaning that matches the speakers style and intent.

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a text that most closely matches the speaker's meaning,
while still giving clear and eloquent English. Give the best approximation or contextual guess if the transcript is difficult or unclear. Make no comme

In [207]:
original_lines

'<8:Có những câu chuyện người ta nói về thiền rất là thú vị>\n<9:Đó là có một người thân chủ của một nhà hàng kia>\n<10:Ông vô tiệm ăn đó>\n<11:Người ta bưng lên những món ăn rất là ngon lành>\n<12:Một bàn đầy>\n<13:Thế rồi ông gọi cái người chủ tiệm lên>\n<14:Ông nói là ông hãy nếm thử thức ăn của ông coi ra làm sao>\n<15:Và cái vị chủ tiệm đó rất là kinh ngại>\n<16:Thứ nhất là đây là một người thân chủ>\n<17:Đến ăn ở đây rất là thường xuyên>\n<18:Thứ hai đó là những đầu bếp của nhà hàng này>\n<19:Là những đầu bếp giỏi nhất>\n<20:Và chế tác những thức ăn rất là ngon>\n<21:Thế nhưng mà hôm nay cái ông thân chủ này>\n<22:Rất là người bạn quen thân này>\n<23:Cũng gọi những món ăn thường ngày ông rất thích thú>\n<24:Mà tại sao ông lại bắn dối như thế này>\n<25:Thế rồi ông chủ nhà hỏi chứ anh không hài lòng về món chi>\n<26:Có phải nó mặn quá không?>\n<27:Không>\n<28:Có phải cách trình bài không đẹp>\n<29:Cho nên anh không hài lòng không?>\n<30:Không>\n<31:Có phải những thức uống không có 

In [237]:
for i in section_range:
    original_lines = unwrap_lines(sections_original[i], number=True).split('\n')
    translated_lines = unwrap_lines(sections_translated[i], number=True).split('\n')

    # Get the maximum length of the original lines
    max_original_len = max(len(line) for line in translated_lines)
    
    # Format output with columns
    for j, (o_l, t_l) in enumerate(zip(original_lines, translated_lines)):
        print(f"{j:>4}: {t_l:<{max_original_len + 5}}<<<   {o_l}")
    # print(section_original[i])
    # print(section_translated[i])

   0: [Bell]                                                                                 <<<   Chuông
   1: [Bell]                                                                                 <<<   Chuông
   2: [Bell]                                                                                 <<<   Chuông
   3: Homage to the Blessed One, the Enlightened One, the Worthy One, Shakyamuni Buddha.     <<<   Nam Mô Đức Bộ Sưu Bụt Thích Ca Mô Đi
   4: Respectfully addressing all Venerable Monks and Nuns present,                          <<<   Kính Bạch Hiện Tiền Chư Tông Đức Tăng Ni
   5: Dear community,                                                                        <<<   Kính Thưa Đại Chúng
   6: Today is August 4, 2024.                                                               <<<   Hôm nay là ngày 4 tháng 8, 2024
   0: There are stories people tell about meditation that are quite interesting.     <<<   Có những câu chuyện người ta nói về thiền rất là thú vị
   1: The

In [183]:
print(postprocess_translate_vi.format(section_title=section.title_en))

You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

You are translating a section titled 'Conclusion' from a Dharma talk offered by a Venerable Vietnamese Monastic at Deer Park Monastery in California.

Common opening salutations are "Namo Shakyamuni Buddha," "Dear Noble Community," which may be expressed partly in Sanskrit, and partly in Vietnamese. 

Some transcriptions may be from sounds such as a bell. These can be marked as [Bell].

You may have to infer the speaker's intent in order to correct transcription or speaking errors and to generate a text that most closely matches the speaker's meaning,
while still giving clear and eloquent English.

Your goal is to translate the text and to correct errors (logical, transcription, or grammatical). 
You must faithfully capture the speaker's style and presentation while creating a meaningful flow using common, clear, and typical English. 
Translate faithfully and as carefully as possible. 



In [166]:
section_object.talk_summary

'In this Dharma Talk, the speaker explores the concepts of internal happiness, spiritual growth, and the cultivation of wisdom and compassion. The talk emphasizes that true happiness arises from inner conditions rather than external factors. It underscores the importance of spiritual discipline in developing mindfulness, wisdom, and unconditional love. The talk also suggests practical methods for cultivating inner peace and spiritual awareness, such as mindful breathing and self-observation.'

In [170]:
section_object.model_dump_json()

'{"talk_summary":"In this Dharma Talk, the speaker explores the concepts of internal happiness, spiritual growth, and the cultivation of wisdom and compassion. The talk emphasizes that true happiness arises from inner conditions rather than external factors. It underscores the importance of spiritual discipline in developing mindfulness, wisdom, and unconditional love. The talk also suggests practical methods for cultivating inner peace and spiritual awareness, such as mindful breathing and self-observation.","sections":[{"title_vi":"Lời mở đầu","title_en":"Introduction","start_line":1,"end_line":7},{"title_vi":"Câu chuyện về khách hàng và chủ tiệm","title_en":"The Story of the Customer and the Restaurant Owner","start_line":8,"end_line":37},{"title_vi":"Bài học từ sự thiếu phương tiện","title_en":"Lesson from Lack of Utensils","start_line":38,"end_line":61},{"title_vi":"Hạnh phúc nội tại so với điều kiện bên ngoài","title_en":"Inner Happiness versus External Conditions","start_line":6

In [172]:
write_text_to_file(section_output_path, section_object.model_dump_json())

In [ ]:
postprocess_text(transcript_path, post_output_path, postprocess_format_en, batch=False)

2024-12-09 19:29:42,444 - tnh.video_processing - INFO - Starting postprocessing of file:
	processed_videos/video_transcriptions/'Awakening Together' Family Retreat Dharma Talk ｜ Sister Kính Nghiêm ｜ 2024-06-27/'Awakening Together' Family Retreat Dharma Talk ｜ Sister Kính Nghiêm ｜ 2024-06-27.txt
2024-12-09 19:29:42,445 - tnh.gpt_interface - DEBUG - Starting chat completion with response_object=None and max_tokens=9235...
2024-12-09 19:31:38,558 - tnh.video_processing - DEBUG - Full completion:
ChatCompletion(id='chatcmpl-AclEM7dUg5T6DS1rQYMZwsbPvfizZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="<section>\n<title>Engaging the Children</title>\n\nI think all the tweens or teens are going to leave the hall. What about the children? Do the children have a program this morning? Can you tell us what you're going to do this morning? Do you know? No? Does it include playing? How have you enjoyed the program so far? It's good? It's good? 